In [6]:
# import libraries
import matplotlib.pyplot as plt
import cv2
import os, glob
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats


In [ ]:
def Main():
    # Note 1: need to manually label (i.e. create masks for yellow lane vs. road and white lane vs. road) 
    # for first k frames.
    k = 10
    
    # Note 2: need to edit file names for data (RGB images = rgbImage+num.jpg, mask = 'mask'+color+'Img'+num.jpg)
    
    # read in previous k images (EDIT PATH)
    listOfAllImages = [plt.imread(path) for path in glob.glob('laneData/rgbImage*.jpg')]
    frames = listOfAllImages[0:k]
    
    # read in previous k masks (EDIT PATH)
    # note: these masks will be binary (0,1) 1 => lane, 0 => road
    listOfAllYellowMasks = [plt.imread(path) for path in glob.glob('laneData/maskYellowImg*.jpg')]
    imageYellowMasks = listOfAllYellowMasks[0:k]
    listOfAllWhiteMasks = [plt.imread(path) for path in glob.glob('laneData/maskWhiteImg*.jpg')]
    imageWhiteMasks = listOfAllWhiteMasks[0:k]
       
    currIdx = 0    
    for img in frames:
        # remove everything above vanishing point?
        
        # find w_max for yellow and white lanes
        w_yellow, w_white = findW(frames, imageYellowMasks, imageWhiteMasks)
        
        # apply weights to current image
        grayImg_yellow = convertToGray(w_yellow, img)
        grayImg_white = convertToGray(w_white, img)
        
        # compute mean of lane vs. road for each yellow and white lanes
        m_l_y
        m_r_y
        m_l_w
        m_r_w
        d_y
        d_w
        
        # apply adaptive canny edge
        canny_y = adaptiveCanny(w_yellow, m_l_y, m_r_y, d_y, grayImg_yellow)
        canny_w = adaptiveCanny(w_white, m_l_w, m_r_w, d_w, grayImg_white)
        
        # combine both images with OR operation
        combined_img = cv2.bitwise_or(canny_y, canny_w)
        
        # HT
        ht = cv2.HoughLinesP(combined_img, rho=1, theta=np.pi/180, threshold=20, minLineLength=20, maxLineGap=300)
        
        # ROI
        
        # Curve-fit
        
        # increment k and update frames and masks
        currIdx = currIdx + 1
        k = k + 1
        frames = listOfAllImages[currIdx:k]
        imageYellowMasks = imageYellowMasks[currIdx:k]
        imageWhiteMasks = imageWhiteMasks[currIdx:k]

In [ ]:
def findW(frames, imageYellowMasks, imageWhiteMasks):
    colors = ['Yellow', 'White']
    
    w_yellow = []
    w_white = []
    
    for c in colors:
        # create training data
        X, y = createTrainingData(frames, "image{}Masks".format(c).replace("'",""))
        
        # compute gradient-enhancing vectors
        w_max = applyLDA(X, y)
        
        if c == 'Yellow':
            w_yellow = w_max
        else:
            w_white = w_max
        
    return w_yellow, w_white
    

In [2]:
# this function creates the training data for class c for frame t.
# k = number of previous frames (default set arbitrarily to 10)
# list_of_RGB_Images = input images
# imagesMasks = correspond to either yellow lane vs. road or white lane vs. road
def createTrainingData(list_of_RGB_Images, imageMasks):
    # Create a list of all RGB values
    R = []
    G = []
    B = []
    for img in list_of_RGB_Images:
        # reshape each color val into an array
        currR = img[:,:,0].reshape(np.shape(img[:,:,0])[0]*np.shape(img[:,:,0])[1]) 
        currG = img[:,:,1].reshape(np.shape(img[:,:,1])[0]*np.shape(img[:,:,1])[1])
        currB = img[:,:,2].reshape(np.shape(img[:,:,2])[0]*np.shape(img[:,:,2])[1])
        
        # concatenate with R, G, and B
        R = np.concatenate((R, currR))
        G = np.concatenate((G, currG))
        B = np.concatenate((B, currB))
        
    y = []
    for mask in imageMasks:
        currY = mask.reshape(np.shape(mask)[0]*np.shape(mask)[1]) 
        y = np.concatenaute((y, currY))
        
    # create training data to return shape(num total pixels, 3(R,G,B))
    X = np.zeros((len(R), 3))
    X[:,0] = R
    X[:,1] = G
    X[:,2] = B

    return X, y


In [4]:
# this function fits data using LDA to find conversion vector from previous images (X,y)
# then applies learned weights to current image (currImage)
def applyLDA(X, y):
    # create classifier object
    clf = LinearDiscriminantAnalysis()

    # fit classifier to data
    clf.fit(X, y)
    
    # return learned weights (i.e. the gradient-enhancing conversion vector)
    return clf.coef_

In [3]:
# this function converts an image with the calculated gradient-enhancing vector
def convertToGray(w, img):
    grayImg = (w[0]*img[:,:,0] + w[1]*img[:,:,1] + w[2]*img[:,:,2]).astype(np.uint8)
    return grayImg

In [7]:
# this function computes two threshold values to determine edges
def adaptiveCanny(w, m_l, m_r, d, image):
    th_L = abs(np.dot(w, m_l) - np.dot(w, m_r))
    th_S = max(abs(np.dot(w, m_l) - d), abs(np.dot(w, m_r) - d))
    
    return cv2.Canny(image, th_S, th_L)